In [ ]:
<!--

#⚽ 01 — EDA Brasileirão (CBF 2003–2023)

Este notebook faz a **análise exploratória (EDA)** e validações básicas de qualidade do dataset da CBF (via Base dos Dados).

**Objetivos:**
- Validar cobertura por ano/rodada
- Checar nulos/duplicatas e consistência de placares
- Criar target `resultado` (H/D/A)
- Exportar uma base mínima e limpa em `data/interim` para uso nos próximos notebooks

**Saída:**
- `data/interim/cbf_2003_2024.parquet` (somente colunas essenciais)

-->

In [1]:
from google.colab import drive
drive.mount("/content/drive")

import pandas as pd
import numpy as np
from pathlib import Path

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 200)

PROJECT = Path("/content/drive/MyDrive/DataProjects/BRMP-Brazilian-Match-Prediction")
RAW = PROJECT / "data" / "raw"
INTERIM = PROJECT / "data" / "interim"
INTERIM.mkdir(parents=True, exist_ok=True)

RAW, INTERIM


Mounted at /content/drive


(PosixPath('/content/drive/MyDrive/DataProjects/BRMP-Brazilian-Match-Prediction/data/raw'),
 PosixPath('/content/drive/MyDrive/DataProjects/BRMP-Brazilian-Match-Prediction/data/interim'))

In [2]:
FILE = RAW / "brasileirao_serie_a_2003_2024.csv"
assert FILE.exists(), f"Arquivo não encontrado: {FILE}"

df = pd.read_csv(FILE)

# Parse + ordenação
df["data"] = pd.to_datetime(df["data"], errors="coerce")
invalid_dates = df["data"].isna().sum()
assert invalid_dates == 0, f"Datas inválidas: {invalid_dates}"

df = df.sort_values("data").reset_index(drop=True)

df.shape, df.head(3)


((8453, 35),
    ano_campeonato       data  rodada                             estadio arbitro  publico  publico_max time_mandante time_visitante tecnico_mandante tecnico_visitante  colocacao_mandante  \
 0            2003 2003-03-29       1  Estádio Brinco de Ouro da Princesa     NaN      NaN          NaN       Guarani  Vasco da Gama              NaN               NaN                 NaN   
 1            2003 2003-03-29       1                    Arena da Baixada     NaN      NaN          NaN   Atlético-PR         Grêmio              NaN               NaN                 NaN   
 2            2003 2003-03-30       1  Estádio Governador Magalhães Pinto     NaN      NaN          NaN      Cruzeiro    São Caetano              NaN               NaN                 NaN   
 
    colocacao_visitante  valor_equipe_titular_mandante  valor_equipe_titular_visitante  idade_media_titular_mandante  idade_media_titular_visitante  gols_mandante  gols_visitante  \
 0                  NaN                

In [3]:
print("Shape:", df.shape)
display(df.head(5))

print("\n--- INFO ---")
display(df.info())

print("\n--- Cobertura por ano (jogos) ---")
games_by_year = df.groupby("ano_campeonato").size()
display(games_by_year)

print("\n--- Rodadas por ano (número de rodadas distintas) ---")
rounds_by_year = df.groupby("ano_campeonato")["rodada"].nunique().sort_index()
display(rounds_by_year)

print("\n--- Rodadas de 2024 (lista) ---")
rounds_2024 = sorted(df.loc[df["ano_campeonato"] == 2024, "rodada"].dropna().unique())
print("Total rodadas distintas em 2024:", len(rounds_2024))
print(rounds_2024)

print("\n--- Contagem de jogos por rodada em 2024 ---")
games_2024_by_round = (
    df[df["ano_campeonato"] == 2024]
    .groupby("rodada")
    .size()
    .rename("jogos")
    .to_frame()
)
display(games_2024_by_round)

print("\n--- % Nulos (top 15) ---")
null_pct = (df.isna().mean().sort_values(ascending=False).head(15))
display(null_pct)

print("\n--- Duplicatas lógicas (data + mandante + visitante) ---")
dup_mask = df.duplicated(subset=["data", "time_mandante", "time_visitante"], keep=False)
print("Total duplicatas lógicas:", int(dup_mask.sum()))
if dup_mask.any():
    display(df.loc[dup_mask, ["data", "time_mandante", "time_visitante", "gols_mandante", "gols_visitante"]].head(10))

print("\n--- Sanity placares ---")
print("Gols mandante nulos:", df["gols_mandante"].isna().sum())
print("Gols visitante nulos:", df["gols_visitante"].isna().sum())
print("Min/Max gols mandante:", (df["gols_mandante"].min(), df["gols_mandante"].max()))
print("Min/Max gols visitante:", (df["gols_visitante"].min(), df["gols_visitante"].max()))


Shape: (8453, 35)


,ano_campeonato,data,rodada,estadio,arbitro,publico,publico_max,time_mandante,time_visitante,tecnico_mandante,tecnico_visitante,colocacao_mandante,colocacao_visitante,valor_equipe_titular_mandante,valor_equipe_titular_visitante,idade_media_titular_mandante,idade_media_titular_visitante,gols_mandante,gols_visitante,gols_1_tempo_mandante,gols_1_tempo_visitante,escanteios_mandante,escanteios_visitante,faltas_mandante,faltas_visitante,chutes_bola_parada_mandante,chutes_bola_parada_visitante,defesas_mandante,defesas_visitante,impedimentos_mandante,impedimentos_visitante,chutes_mandante,chutes_visitante,chutes_fora_mandante,chutes_fora_visitante
0,2003,2003-03-29,1,Estádio Brinco de Ouro da Princesa,NaN,NaN,NaN,Guarani,Vasco da Gama,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2003,2003-03-29,1,Arena da Baixada,NaN,NaN,NaN,Atlético-PR,Grêmio,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2003,2003-03-30,1,Estádio Governador Magalhães Pinto,NaN,NaN,NaN,Cruzeiro,São Caetano,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2003,2003-03-30,1,Estádio Jornalista Mário Filho,NaN,NaN,NaN,Flamengo,Coritiba FC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2003,2003-03-30,1,Estádio Governador Plácido Castelo,NaN,NaN,NaN,Fortaleza,EC Bahia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



--- INFO ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8453 entries, 0 to 8452
Data columns (total 35 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   ano_campeonato                  8453 non-null   int64         
 1   data                            8453 non-null   datetime64[ns]
 2   rodada                          8453 non-null   int64         
 3   estadio                         8442 non-null   object        
 4   arbitro                         6743 non-null   object        
 5   publico                         6742 non-null   float64       
 6   publico_max                     4181 non-null   float64       
 7   time_mandante                   8453 non-null   object        
 8   time_visitante                  8453 non-null   object        
 9   tecnico_mandante                6299 non-null   object        
 10  tecnico_visitante               6299 non-null   object    

None


--- Cobertura por ano (jogos) ---


...


--- Rodadas de 2024 (lista) ---
Total rodadas distintas em 2024: 29
[np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(12), np.int64(13), np.int64(14), np.int64(15), np.int64(16), np.int64(17), np.int64(18), np.int64(19), np.int64(20), np.int64(21), np.int64(22), np.int64(23), np.int64(24), np.int64(25), np.int64(26), np.int64(27), np.int64(28), np.int64(29), np.int64(30), np.int64(31)]

--- Contagem de jogos por rodada em 2024 ---


,jogos
rodada,
1,10
2,10
3,10
4,10
5,10
6,10
7,10
8,10
9,10



--- % Nulos (top 15) ---


,0
chutes_fora_visitante,0.753105
defesas_visitante,0.753105
defesas_mandante,0.753105
faltas_mandante,0.753105
faltas_visitante,0.753105
chutes_bola_parada_visitante,0.753105
chutes_fora_mandante,0.753105
impedimentos_visitante,0.753105
impedimentos_mandante,0.753105
chutes_visitante,0.753105



--- Duplicatas lógicas (data + mandante + visitante) ---
Total duplicatas lógicas: 0

--- Sanity placares ---
Gols mandante nulos: 1
Gols visitante nulos: 1
Min/Max gols mandante: (0.0, 7.0)
Min/Max gols visitante: (0.0, 7.0)


In [4]:
# Remove linhas sem placar (se existirem)
df = df.dropna(subset=["gols_mandante", "gols_visitante"]).copy()

def compute_result(row):
    if row["gols_mandante"] > row["gols_visitante"]:
        return "H"
    elif row["gols_mandante"] < row["gols_visitante"]:
        return "A"
    return "D"

df["resultado"] = df.apply(compute_result, axis=1)

print("--- Distribuição geral (H/D/A) ---")
display(df["resultado"].value_counts())

print("\n--- Distribuição por ano (%) ---")
dist_year = (
    df.groupby(["ano_campeonato", "resultado"])
      .size()
      .groupby(level=0)
      .apply(lambda s: s / s.sum())
      .unstack()
      .fillna(0)
)
display(dist_year.tail(10))


--- Distribuição geral (H/D/A) ---


,count
resultado,
H,4182
D,2248
A,2022



--- Distribuição por ano (%) ---


,resultado,A,D,H
ano_campeonato,ano_campeonato,,,
2015,2015,0.234211,0.239474,0.526316
2016,2016,0.218997,0.248021,0.532982
2017,2017,0.289474,0.271053,0.439474
2018,2018,0.178947,0.289474,0.531579
2019,2019,0.257895,0.257895,0.484211
2020,2020,0.265789,0.284211,0.450000
2021,2021,0.244737,0.297368,0.457895
2022,2022,0.273684,0.284211,0.442105
2023,2023,0.273684,0.257895,0.468421


In [5]:
df = df[df["ano_campeonato"] <= 2023].copy()


In [6]:
# Base mínima para os próximos passos (modelagem e comparações)
cols = [
    "data",
    "ano_campeonato",
    "rodada",
    "time_mandante",
    "time_visitante",
    "gols_mandante",
    "gols_visitante",
    "resultado",
]

cbf_base = df[cols].copy()

out = INTERIM / "cbf_2003_2023.parquet"
cbf_base.to_parquet(out, index=False)

print("✅ Export concluído:", out)
print("Shape:", cbf_base.shape)
display(cbf_base.head(5))


✅ Export concluído: /content/drive/MyDrive/DataProjects/BRMP-Brazilian-Match-Prediction/data/interim/cbf_2003_2023.parquet
Shape: (8169, 8)


,data,ano_campeonato,rodada,time_mandante,time_visitante,gols_mandante,gols_visitante,resultado
0,2003-03-29,2003,1,Guarani,Vasco da Gama,4.0,2.0,H
1,2003-03-29,2003,1,Atlético-PR,Grêmio,2.0,0.0,H
2,2003-03-30,2003,1,Cruzeiro,São Caetano,2.0,2.0,D
3,2003-03-30,2003,1,Flamengo,Coritiba FC,1.0,1.0,D
4,2003-03-30,2003,1,Fortaleza,EC Bahia,0.0,0.0,D


In [9]:
<!--
## 🧾 Sumário Final — EDA Brasileirão (CBF 2003–2023)

Este notebook realizou a análise exploratória e validações básicas do dataset histórico do Campeonato Brasileiro (Série A), considerando **exclusivamente o período de 2003 a 2023**.

### 🔍 Principais verificações realizadas
- Leitura e ordenação temporal dos dados
- Validação de datas, placares e consistência geral
- Análise de cobertura por ano e rodadas
- Avaliação de valores ausentes e duplicatas lógicas
- Criação do target **resultado** (`H`, `D`, `A`)

### 📊 Cobertura temporal
- Período analisado: **2003 → 2023**
- Cobertura anual consistente, com aproximadamente **380 jogos por temporada** nos anos de pontos corridos
- Não foram identificadas inconsistências graves de duplicidade de partidas

### 🎯 Target (Resultado da Partida)
O target foi definido a partir do placar final:
- **H** — Vitória do mandante
- **D** — Empate
- **A** — Vitória do visitante

A distribuição do target mostrou-se **realista e estável ao longo dos anos**, com maior incidência de vitórias do mandante, padrão esperado no futebol brasileiro.

### ⚠️ Observações importantes
- A temporada **2024 foi explicitamente removida** deste notebook para evitar:
  - dados incompletos
  - rodadas faltantes
  - possíveis distorções na análise exploratória
- A resolução de diferenças entre fontes e tratamento de temporadas recentes é feita no notebook
  **`02_compare_sources.ipynb`**

### 💾 Saída do Notebook
Foi gerada uma base mínima e limpa contendo apenas colunas essenciais:

- `data`
- `ano_campeonato`
- `rodada`
- `time_mandante`
- `time_visitante`
- `gols_mandante`
- `gols_visitante`
- `resultado`

📦 Arquivo exportado: data/interim/cbf_2003_2023.parquet

### ➡️ Próximo passo
O próximo notebook do pipeline é:

**`02_compare_sources.ipynb`**
Responsável por:
- Comparar a base da CBF com `football-data.co.uk`
- Resolver divergências temporais (±1 dia)
- Definir a fonte final para modelagem preditiva

-->


SyntaxError: invalid character '→' (U+2192) (ipython-input-3119272192.py, line 14)